# Вебинар 5. Ранжирование и Гибридные рекоммендательные системы

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k
    
# from metrics import precision_at_k, recall_at_k
from utils import prefilter_items

C:\Users\konst\AppData\Local\Programs\Python\Python37\lib\site-packages\lightfm\_lightfm_fast.py:10: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  "LightFM was compiled without OpenMP support. "


In [2]:
data = pd.read_csv('./lessons/web2/retail_train.csv')
item_features = pd.read_csv('./lessons/web2/product.csv')
user_features = pd.read_csv('./lessons/web2/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


## 1. Filter items

In [5]:
data_train.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [6]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


# 2. Prepare data set

## 2.1 Prepare csr train matrix

In [36]:

user_item_matrix = pd.pivot_table(
    data_train,
    index='user_id',
    columns='item_id',
    values='quantity',
    aggfunc='count',
    fill_value=0,
)

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 2.2 Prepare CSR test matrix

In [37]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(
    data_test,
    index='user_id',
    columns='item_id',
    values='quantity',
    aggfunc='count',
    fill_value=0,
)

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [38]:
test_user_item_matrix.head(2)

item_id,32392,34873,42852,43094,44522,45507,49812,53516,58612,62804,...,17284297,17284423,17291184,17291665,17320698,17320911,17321092,17328261,17329749,17330255
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

## 3. Prepare user and item features

In [40]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
user_feat.shape

(2499, 7)

In [42]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [43]:
item_feat.shape

(86865, 6)

## Encoding features

In [44]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [45]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
item_feat_lightfm.head(2)

,manufacturer_1,manufacturer_2,manufacturer_3,manufacturer_4,manufacturer_5,manufacturer_6,manufacturer_8,manufacturer_9,manufacturer_10,manufacturer_11,...,curr_size_of_product_TWIN,curr_size_of_product_TWIN PACK,curr_size_of_product_UNDER 15LB,curr_size_of_product_X-LARGE,curr_size_of_product_X-LG,curr_size_of_product_XL,curr_size_of_product_XL/3PK,curr_size_of_product_XL/6PK,curr_size_of_product_XLG,curr_size_of_product_XLRG
item_id,,,,,,,,,,,,,,,,,,,,,
25671,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26081,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Init model

In [47]:
model = LightFM(
    no_components=40,
    loss='warp',
    learning_rate=0.05, 
    item_alpha=0.1,
    user_alpha=0.1, 
    random_state=42
)

## Train

In [48]:
model.fit(
    (sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
    sample_weight=coo_matrix(user_item_matrix),
    user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
    item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
    epochs=1, 
    num_threads=4,
    verbose=False,
) 

# Test precision

In [49]:
test_precision = precision_at_k(
    model,
    csr_matrix(test_user_item_matrix).tocsr(),
    user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
    item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
    k=5
)

test_precision.mean()

0.0018618326

# Домашнее задание

In [50]:
import datetime
from sklearn.model_selection import ParameterGrid


def grid_search(param_grid: dict) -> dict:
    time: datetime.datetime = datetime.datetime.now()
    results: dict = dict()
    grid: list = list(ParameterGrid(param_grid))
    cnt: int = 0
    for params in grid:
        cnt += 1
        results[cnt] = params
        
        model = LightFM(
            random_state=42,
            **params,
        )
        model.fit(
            (sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
            sample_weight=coo_matrix(user_item_matrix),
            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
            epochs=1, 
            num_threads=4,
            verbose=False,
        )

        precision = precision_at_k(
            model,
            csr_matrix(test_user_item_matrix).tocsr(),
            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
            k=5
        )

        results[cnt].update({'precision_at_k': precision.mean()})

        print(f'Iteration {cnt} from {len(grid)} {datetime.datetime.now()} (spent {datetime.datetime.now() - time})')
        print(results[cnt])
        time = datetime.datetime.now()

    return results

param_grid = {
    'no_components': [10, 20, 40],
    'loss': ['bpr', 'warp'],
    'learning_rate': [0.01, 0.05, 0.1], 
    'item_alpha': [0, 0.5],
    'user_alpha': [0, 0.5],
}
results = grid_search(param_grid)

Iteration 1 from 72 2021-06-19 23:17:19.585916 (spent 0:01:51.991960)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'bpr', 'no_components': 10, 'user_alpha': 0, 'precision_at_k': 0.00039196474}
Iteration 2 from 72 2021-06-19 23:19:01.671261 (spent 0:01:42.076342)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'bpr', 'no_components': 10, 'user_alpha': 0.5, 'precision_at_k': 0.00029397354}
Iteration 3 from 72 2021-06-19 23:21:12.514116 (spent 0:02:10.841804)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'bpr', 'no_components': 20, 'user_alpha': 0, 'precision_at_k': 0.0009799118}
Iteration 4 from 72 2021-06-19 23:23:25.949052 (spent 0:02:13.432939)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'bpr', 'no_components': 20, 'user_alpha': 0.5, 'precision_at_k': 0.001567859}
Iteration 5 from 72 2021-06-19 23:26:36.269840 (spent 0:03:10.319802)
{'item_alpha': 0, 'learning_rate': 0.01, 'loss': 'bpr', 'no_components': 40, 'user_alpha': 0, 'precision_at_k': 0.0011758942}
Iteration 6 fro

In [51]:
pd.DataFrame.from_dict(results, orient='index').sort_values(by='precision_at_k', ascending=False)

,item_alpha,learning_rate,loss,no_components,user_alpha,precision_at_k
46,0.5,0.01,warp,20,0.5,0.006369
68,0.5,0.10,warp,10,0.5,0.005194
34,0.0,0.10,warp,20,0.5,0.004900
36,0.0,0.10,warp,40,0.5,0.004802
49,0.5,0.05,bpr,10,0.0,0.004802
...,...,...,...,...,...,...
37,0.5,0.01,bpr,10,0.0,0.000490
38,0.5,0.01,bpr,10,0.5,0.000490
1,0.0,0.01,bpr,10,0.0,0.000392
2,0.0,0.01,bpr,10,0.5,0.000294
